This notebook shows a couple of examples to include the Bing Search Plugin for Semantic Kernel.  

In [16]:
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.FileExtensions, 8.0.1"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.17.2"
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.18.1-alpha"


using Microsoft.Extensions.Configuration;
using System.IO;

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Plugins.Web;

public static var config = new ConfigurationBuilder()
          .AddJsonFile(Path.GetFullPath("secrets.json"), optional: false, reloadOnChange: true)
          .Build();

public static string OpenAIKey =  config["openai-key"];
public static string BingSearchKey = config["bing-key"];

Installed Packages Microsoft.Extensions.Configuration, 8.0.0 Microsoft.Extensions.Configuration.FileExtensions, 8.0.1 Microsoft.Extensions.Configuration.Json, 8.0.0 Microsoft.SemanticKernel, 1.17.2 Microsoft.SemanticKernel.Plugins.Web, 1.18.1-alpha

Set up Semantic Kernel

In [31]:
static string ModelId = "gpt-4o-mini";

// Create a kernel with OpenAI chat completion
var builder = Kernel.CreateBuilder()
.AddOpenAIChatCompletion(ModelId, OpenAIKey);

Kernel kernel = builder.Build();

Configure the Bing Search Plugin.  

> Note that at this time the BingConnector gives an error warning indicating it is experimental and the pragma is required to compile the code.

In [32]:
#pragma warning disable SKEXP0050 
var bingConnector = new BingConnector(BingSearchKey);

kernel.ImportPluginFromObject(new WebSearchEnginePlugin(bingConnector), "bing");


This example shows the current organizers and seems to work correctly.  This was pulled from an example I found.

In [34]:
var prompt = "Who are the organizers for the Boston Azure meetup?";
OpenAIPromptExecutionSettings settings = new() 
    { 
        ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions, 
        Temperature = 0.7f,
        MaxTokens = 250
    };
 var funcresult = await kernel.InvokePromptAsync(prompt, new KernelArguments(settings));
 funcresult

Below does not work as well as I'd hoped.  The hyperlinks do not take you to the articles and the articles are very general.  I think I could tune the prompt to get better articles, but after reading the acceptable usage of the Bing Search API, it appears I would be in violation if I used the search API in the way I had envisioned it.

In [43]:
var query = "list the hyperlinks and summaries for the top 10 news articles related to .NET Programing";

var result = await kernel.InvokePromptAsync(query, new KernelArguments(settings));

Console.WriteLine($"\nANSWER: \n\n{result}");


ANSWER: 

Here are the top 10 news articles related to .NET Programming:

1. **[Announcing the Azure AI Inference SDK for .NET!](https://devblogs.microsoft.com/dotnet/)**  
   August 13, 2024. This SDK provides easy access to a comprehensive suite of AI models in the Azure AI Model Catalog for inference tasks, enabling integration of AI into applications.

2. **[What’s next in .NET?](https://dotnet.microsoft.com/en-us/next)**  
   Stay up to date with the .NET 9 preview, the next version of .NET, including recent updates and features for developers.

3. **[What’s new in .NET 9 Preview 7?](https://devblogs.microsoft.com/dotnet/dotnet-9-preview-7/)**  
   Overview of new features and breaking changes in .NET 9 Preview 7, encouraging feedback from the developer community.

4. **[What’s new in .NET 9 | Microsoft Learn](https://learn.microsoft.com/en-us/dotnet/core/whats-new/dotnet-9/overview)**  
   Details on new cryptography features, assembly building, and other enhancements in
